In [ ]:
#Run this cell to open ssh connection tunnel to current colab session via ngrok.
#You need to be registered on ngrok website and provide your authtoken.
#After establishing connection credentials will be stored at /content/credentials file.
import urllib.request, json, getpass, secrets, time

#Generate root password
password = secrets.token_urlsafe(32)

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Add user and password to credentials file
! echo user=$(whoami) > ./credentials
! echo password=$password >> ./credentials
! echo $(whoami):$password | chpasswd

#Configure sshd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
authtoken = getpass.getpass(
  prompt="Copy authtoken from https://dashboard.ngrok.com/auth here:\n"
)

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
time.sleep(2)

#Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  host, port = data['tunnels'][0]['public_url'][6:].split(':')

#Add host and port to credentials file
! echo host=$host >> ./credentials
! echo port=$port >> ./credentials